In [1]:
import numpy as np
import re
import os
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import calendar
import warnings
import datetime
import seaborn as sns
from datetime import date
from matplotlib.colors import LinearSegmentedColormap
warnings.filterwarnings('ignore')
import shutil
import os
import time


In [2]:
Start_D = input('Provide Start Date in format (YYYY-MM-DD):')
End_D = input('Provide End Date in format (YYYY-MM-DD):')

Provide Start Date in format (YYYY-MM-DD):2022-12-01
Provide End Date in format (YYYY-MM-DD):2022-12-31


In [3]:
#Select the Product/Service for which you want to observe the data
#Option
print("Select the option --> UDP or CLARITY or CAMS or UDP2 or API or DEVOPS or DAP or INQUISITO or PKI")
prod = input('Product:')

Select the option --> UDP or CLARITY or CAMS or UDP2 or API or DEVOPS or DAP or INQUISITO or PKI
Product:


In [4]:
print("Select the option --> Year(YYYY) in range between 2021-2029")
while True:
    try:
        year = input('Year(YYYY): ')
        if not re.match("^202[1-9]$", year):
            raise ValueError   
        break
    except ValueError:
            print('Error! Only provide in Year Format yyyy in range between 2021-2029')

Select the option --> Year(YYYY) in range between 2021-2029
Year(YYYY): 2022


In [5]:
month = []
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

while True:
    try:
        n = int(input("Enter number of Months in Integer (1 or 2): "))
        if n < 1 or n > 2:
            raise ValueError #this will send it to the print message and back to the input option
        break
    except ValueError:
        print('Please enter an integer 1 or 2')
        
# iterating till the range
print("Month in format Mmm")
print("For example January as Jan")
for i in range(0, n):
    while True:
        try:
            ele = input("Month-{} in format Mmm: ".format(i+1))
            #if not re.match("^[A-Z][a-z][a-z]$", ele):
            #    raise ValueError
            if ele not in months:
                raise ValueError
            break
        except ValueError:
            print('Error! Only provide in Month Format Mmm!')
    month.append(ele) # adding the element
print(month)

Enter number of Months in Integer (1 or 2): 1
Month in format Mmm
For example January as Jan
Month-1 in format Mmm: Dec
['Dec']


In [6]:
file_list = []

for i in range(len(month)):
    file_list.append("Weekly-Monthly-KPI-{}-{}.xlsx".format(month[i],year))
print("File List:")
for j in range(len(file_list)):
    print(file_list[j])

File List:
Weekly-Monthly-KPI-Dec-2022.xlsx


In [7]:
finalDF = pd.DataFrame()

for file in file_list:
    sheets = pd.ExcelFile(""+file)
    myWorkSheets = sheets.sheet_names
    for sheet in myWorkSheets:
        if sheet.startswith("Pager") or sheet.startswith("BigPanda"):
            df = pd.read_excel(""+file,sheet_name=sheet)
            finalDF = finalDF.append(df)

In [8]:
df11 = finalDF.copy()

In [9]:
df11 = df11.drop(['BigPanda Incident Link','Jira Ticket Updated'], axis = 1)

In [10]:
df11.columns = ['Jira Ticket','Subject','Product','Location','Priority','Customer Impacted','Date Time','Incident End','First Assignment','Actioned Time', 'Ticket created on','TTD','TTE','TTAE','TTA','TTR','Assignee','NAR','RNE','ETS','TSG','RCA','Total Alerts','Incident Start time',
       'Escalated Time']

In [11]:
df11['TTA'].replace(np.nan, 0)

0       0.0
1      68.0
2       0.0
3       0.0
4       0.0
       ... 
97      0.0
98      0.0
99      0.0
100     6.0
101     0.0
Name: TTA, Length: 185, dtype: float64

In [12]:
df11['Date Time'] = pd.to_datetime(df11['Date Time'])
df11['Date'] = df11['Date Time'].dt.strftime('%m-%d-%Y')
df11['Date'] = pd.to_datetime(df11['Date'])
df11['Month'] = pd.DatetimeIndex(df11['Date Time']).month
df11['Day'] = pd.DatetimeIndex(df11['Date Time']).day
df11['Year'] = pd.DatetimeIndex(df11['Date Time']).year
df11['Quarter'] = pd.DatetimeIndex(df11['Date Time']).quarter
df11['Week'] = pd.DatetimeIndex(df11['Date Time']).week
df11['Weekday'] = pd.DatetimeIndex(df11['Date Time']).weekday
df11['Weekday'] = df11['Weekday'].apply(lambda x: calendar.day_name[x])
df11['Dayofyear'] = pd.DatetimeIndex(df11['Date Time']).dayofyear
df11['Weekofyear'] = pd.DatetimeIndex(df11['Date Time']).weekofyear
df11['date_ordinal'] = pd.to_datetime(df11['Date Time']).apply(lambda date: date.toordinal())
df11['Hours'] = df11['Date Time'].dt.hour

In [13]:
df11['TSG'] = df11['TSG'].apply(lambda name : name.upper())
df11['Product'] = df11['Product'].apply(lambda name : name.upper())
df11['Location'] = df11['Location'].apply(lambda name : name.upper())
#df11['Assignee'] = df11['Assignee'].apply(lambda name : name.upper())
df11['Weekday'] = df11['Weekday'].apply(lambda name : name.upper())
df11['NAR'] = df11['NAR'].apply(lambda name : name.upper())
df11['RNE'] = df11['RNE'].apply(lambda name : name.upper())
df11['ETS'] = df11['ETS'].apply(lambda name : name.upper())

In [14]:
df11.Product = df11.Product.str.replace(" ","")
df11.Priority = df11.Priority.str.replace(" ","")
df11.Location = df11.Location.str.replace(" ","")
df11.TSG = df11.TSG.str.replace(" ","")
df11['Jira Ticket'] = df11['Jira Ticket'].str.strip()
df11['Subject'] = df11['Subject'].str.strip()
df11['Customer Impacted'] = df11['Customer Impacted'].str.strip()
df11['RCA'] = df11['RCA'].str.strip()
df11['NAR'] = df11['NAR'].str.strip()
df11['RNE'] = df11['RNE'].str.strip()
df11['ETS'] = df11['ETS'].str.strip()

In [15]:
df11.Product = df11.Product.str.replace("NILE","API")
df11.Product = df11.Product.str.replace("UAM","CAMS")

In [16]:
if Start_D > End_D:
    print("Start Date and End Date will be corrected automatically /n Next time please enter date properly")
    st_dt = End_D
    ed_dt = Start_D
else:
    print("Date Range has been entered properly")
    st_dt = Start_D
    ed_dt = End_D
Start_D = st_dt
End_D = ed_dt
print("Start Date :",Start_D)
print("End Date :",End_D)

Date Range has been entered properly
Start Date : 2022-12-01
End Date : 2022-12-31


In [17]:
if prod == "":
    mask = (df11['Date'] >= Start_D) & (df11['Date'] <= End_D)
else:
    mask = (df11['Date'] >= Start_D) & (df11['Date'] <= End_D) & (df11['Product'] == prod)
filtered_df=df11.loc[mask]
df1 = filtered_df.copy()
df1['Date'] = df1['Date'].dt.strftime('%m-%d-%Y')

In [18]:
df1['TTA'] = df1['TTA'].replace(np.nan, 0)
df1['Escalated Time'] = df1['Escalated Time'].replace(np.nan, 0)
df1['ETS'] = df1['ETS'].replace(np.nan, 0)
df1['TTAE'] = df1['TTAE'].replace(np.nan, 0)

In [19]:
df1 = df1.filter(['Jira Ticket','Subject','Product','Location','Priority','Customer Impacted','Date Time','Incident End','First Assignment','Actioned Time', 'Ticket created on','TTD','TTE','TTAE','TTA','TTR','Assignee','NAR','RNE','ETS','TSG','RCA','Total Alerts','Incident Start time',
       'Escalated Time'])

In [20]:
df1.head()

,Jira Ticket,Subject,Product,Location,Priority,Customer Impacted,Date Time,Incident End,First Assignment,Actioned Time,...,TTR,Assignee,NAR,RNE,ETS,TSG,RCA,Total Alerts,Incident Start time,Escalated Time
0,INC-2640,UDP2 Prod US: Critical - Services Overall - Se...,UDP2,US,P4,No Customer Impact,2022-12-01 08:57:00,2022-12-01 09:02:00,2022-12-01 08:57:00,NaT,...,5.0,Ankit Singh,NO,NO,NO,NO,It was a blip,1,2022-12-01 08:56:00,0
1,INC-2649,UDP2 Prod - prod-us-5g-udp2-1: Kafka Consumer ...,UDP2,US,P4,No Customer Impact,2022-12-02 16:15:00,2022-12-02 17:28:00,2022-12-02 16:16:00,2022-12-02 17:23:00,...,73.0,Varsha Rana,NO,NO,YES,YES,SME updated the monitor,2,2022-12-02 16:15:00,0
2,INC-2657,Clarity Prod JP: CRITICAL - P3 - 5% Subjects w...,CLARITY,JP,P3,24 users (28.23% impacted),2022-12-05 08:14:00,2022-12-05 12:37:00,2022-12-05 08:14:00,NaT,...,263.0,Nitesh Kumar,YES,NO,YES,YES,It was a blip,1,2022-12-05 08:13:00,2022-12-05 09:24:00
3,INC-2661,Clarity Prod JP: CRITICAL - P4 - 0.5% Subjects...,CLARITY,JP,P4,1 user (0.62% Impacted),2022-12-05 17:37:00,2022-12-05 21:37:00,2022-12-05 17:38:00,NaT,...,240.0,Shadab Tamboli,YES,NO,NO,YES,It was a blip,1,2022-12-05 17:37:00,0
4,INC-2664,UDP2 Prod US: Critical - API Gateway - Unable ...,UDP2,US,P4,No Customer Impact,2022-12-06 06:49:00,2022-12-06 06:59:00,2022-12-06 06:49:00,NaT,...,10.0,Pavan K,YES,NO,NO,YES,It was a blip,1,2022-12-06 06:48:00,0


In [21]:
df1["Subject"] = df1["Subject"].apply(lambda x: x.replace("[DE]:", " "))

# TTD Validation

In [31]:
TTD = df1['TTD']
#Check for TTD
for index, value in TTD.items():   #To get the exact position of the error
    if(value > 1):
        print(f"TTD is too high! {index}!", value)
        

In [50]:
print("The following tickets are having High Time to Detect! : ")
df1.loc[(df1['TTD'] > 1)]

The following tickets are having High Time to Detect! : 


,Jira Ticket,Subject,Product,Location,Priority,Customer Impacted,Date Time,Incident End,First Assignment,Actioned Time,...,Assignee,NAR,RNE,ETS,TSG,RCA,Total Alerts,Incident Start time,Escalated Time,TP_3


# TTE Validation

In [33]:
TTE = df1['TTE']
for index, value in TTE.items():   #To get the exact position of the error
    if(value > 1.1):
        print(f"Took time to engage at index {index}!", value)
        

Took time to engage at index 0! 4.0
Took time to engage at index 2! 4.0
Took time to engage at index 8! 2.0
Took time to engage at index 13! 2.0
Took time to engage at index 25! 2.0
Took time to engage at index 29! 2.0
Took time to engage at index 43! 3.0
Took time to engage at index 44! 2.0
Took time to engage at index 49! 2.0
Took time to engage at index 59! 3.0
Took time to engage at index 69! 3.0
Took time to engage at index 80! 5.0


In [51]:
print("The following tickets are having High Time to Engage! : ")
df1.loc[(df1['TTE'] > 3)]

The following tickets are having High Time to Engage! : 


,Jira Ticket,Subject,Product,Location,Priority,Customer Impacted,Date Time,Incident End,First Assignment,Actioned Time,...,Assignee,NAR,RNE,ETS,TSG,RCA,Total Alerts,Incident Start time,Escalated Time,TP_3
0,INC-2640,UDP2 Prod US: Critical - Services Overall - Se...,UDP2,US,P4,No Customer Impact,2022-12-01 08:57:00,2022-12-01 09:02:00,2022-12-01 08:57:00,NaT,...,Ankit Singh,0,NO,0,NO,It was a blip,1,2022-12-01 08:56:00,0,US
2,INC-2642,Clarity Prod EU: CRITICAL - P4 - 0.5% Subjects...,CLARITY,EU,P4,86 user (2.37% Impacted),2022-12-01 13:46:00,2022-12-01 14:49:00,2022-12-01 13:50:00,NaT,...,Varsha Rana,1,NO,0,YES,It was a blip,1,NaN,0,EU
80,INC-2768,UDP2 Prod - : MySQL Ent: Load Exception,UDP2,EU,P4,No Customer Impact,2022-12-18 00:31:00,2022-12-18 00:48:00,2022-12-18 00:36:00,NaT,...,Jainil Lalaji,1,NO,0,NO,It was a blip,2,NaN,0,- : MYSQL ENT


In [35]:
df1['NAR'] = df1['NAR'].replace({'NO': 0, 'YES': 1})
df1['ETS'] = df1['ETS'].replace({'NO': 0, 'YES': 1})

In [36]:
df1['TTA'] = df1['TTA'].apply(np.int64)

In [37]:
#print(df1.dtypes)

# TTA Validation


In [38]:
I = df1.loc[(df1['NAR'] == 0) & (df1['TTA'] == 0)]
J = df1.loc[(df1['NAR'] == 1) & (df1['TTA'] != 0)]
df_append = I.append(J)
df_append

,Jira Ticket,Subject,Product,Location,Priority,Customer Impacted,Date Time,Incident End,First Assignment,Actioned Time,...,TTR,Assignee,NAR,RNE,ETS,TSG,RCA,Total Alerts,Incident Start time,Escalated Time
0,INC-2640,UDP2 Prod US: Critical - Services Overall - Se...,UDP2,US,P4,No Customer Impact,2022-12-01 08:57:00,2022-12-01 09:02:00,2022-12-01 08:57:00,NaT,...,5.0,Ankit Singh,0,NO,0,NO,It was a blip,1,2022-12-01 08:56:00,0
15,INC-2659,[Synthetics] Inquisito Prod US: Api Health,INQUISITO,US,P4,No Customer Impact,2022-12-05 11:35:00,2022-12-06 13:56:00,2022-12-05 11:35:00,NaT,...,1581.0,Smruti Sakhare,0,NO,1,YES,SME made changes to the Datadog monitor,1,NaN,0
7,INC-2672,Clarity Prod JP: CRITICAL - P4 - 0.5% Subjects...,CLARITY,JP,P4,1 user (0.86% Impacted),2022-12-07 02:12:00,2022-12-07 07:01:00,2022-12-07 02:13:00,NaT,...,289.0,Atharva Jagdale,1,NO,0,YES,It was a blip,2,2022-12-07 02:12:00,0


# JIRA Ticket Validation

In [39]:
df1.loc[(df1['Jira Ticket'] == " ")]

,Jira Ticket,Subject,Product,Location,Priority,Customer Impacted,Date Time,Incident End,First Assignment,Actioned Time,...,TTR,Assignee,NAR,RNE,ETS,TSG,RCA,Total Alerts,Incident Start time,Escalated Time


# TTEs Validation

In [42]:
L = df1.loc[(df1['ETS'] == 0) & (df1['Escalated Time'] != 0)]
M = df1.loc[(df1['ETS'] == 1) & (df1['Escalated Time'] == 0)]
N = df1.loc[(df1['ETS'] == 0) & (df1['TTAE'] != 0)]
O = df1.loc[(df1['ETS'] == 1) & (df1['TTAE'] == 0)]

df_append= L.append([M, N, O])
df_append

,Jira Ticket,Subject,Product,Location,Priority,Customer Impacted,Date Time,Incident End,First Assignment,Actioned Time,...,TTR,Assignee,NAR,RNE,ETS,TSG,RCA,Total Alerts,Incident Start time,Escalated Time
1,INC-2649,UDP2 Prod - prod-us-5g-udp2-1: Kafka Consumer ...,UDP2,US,P4,No Customer Impact,2022-12-02 16:15:00,2022-12-02 17:28:00,2022-12-02 16:16:00,2022-12-02 17:23:00,...,73.0,Varsha Rana,0,NO,1,YES,SME updated the monitor,2,2022-12-02 16:15:00,0
15,INC-2659,[Synthetics] Inquisito Prod US: Api Health,INQUISITO,US,P4,No Customer Impact,2022-12-05 11:35:00,2022-12-06 13:56:00,2022-12-05 11:35:00,NaT,...,1581.0,Smruti Sakhare,0,NO,1,YES,SME made changes to the Datadog monitor,1,NaN,0
26,INC-2677,UDP2 Prod - prod-us-5g-udp2-1: Kafka Consumer ...,UDP2,US,P1,Some of US Followers are seeing notification d...,2022-12-07 16:10:00,2022-12-08 11:59:00,2022-12-07 16:10:00,NaT,...,1189.0,Jainil Lalaji,0,NO,1,YES,"SME cordoned all nodes in ""us-central1-f"" and ...",15,NaN,0
40,INC-2702,UDP2 Prod - prod-us-5g-udp2-1: Kafka Consumer ...,UDP2,-,P4,No Customer Impact,2022-12-10 23:15:00,2022-12-10 23:31:00,2022-12-10 23:16:00,NaT,...,16.0,Nitesh Kumar,1,NO,1,YES,It was a blip,2,NaN,0
50,INC-2719,CAMS Prod US: Critical - Kafka Accounts Prima...,CAMS,US,P4,No Customer Impact,2022-12-13 03:42:00,2022-12-13 04:35:00,2022-12-13 03:43:00,NaT,...,53.0,Shadab Tamboli,1,NO,1,NO,It was a blip,1,NaN,0
98,INC-2808,Clarity Prod EU: CRITICAL - P4 - 0.5% Subjects...,CLARITY,EU,P3,214 users (6.7% Impacted),2022-12-21 13:49:00,2022-12-21 15:26:00,2022-12-21 13:50:00,NaT,...,96.0,Varsha Rana,1,NO,1,YES,Pods weren't getting any traffic.,3,NaN,0
100,INC-2812,g5-dexcom-prod-us-5 - Spark Job nimbus-hourl...,UDP,US,P4,No Customer Impact,2022-12-21 22:10:00,2022-12-21 23:52:00,2022-12-21 22:11:00,2022-12-21 22:16:00,...,102.0,Pavan K,0,NO,1,YES,SME has updated the TTL value and restarted th...,1,NaN,0
50,INC-2719,CAMS Prod US: Critical - Kafka Accounts Prima...,CAMS,US,P4,No Customer Impact,2022-12-13 03:42:00,2022-12-13 04:35:00,2022-12-13 03:43:00,NaT,...,53.0,Shadab Tamboli,1,NO,1,NO,It was a blip,1,NaN,0


# TTR Validation

In [43]:
df1.loc[(df1['TTR'] <= 0)]

,Jira Ticket,Subject,Product,Location,Priority,Customer Impacted,Date Time,Incident End,First Assignment,Actioned Time,...,TTR,Assignee,NAR,RNE,ETS,TSG,RCA,Total Alerts,Incident Start time,Escalated Time


In [44]:
df1['TP_1'] = df1.Subject.str.extract(r'Prod (.*):',expand=False)
df1['TP_2'] = df1.Subject.str.extract(r'((?<=prod-)[a-zA-Z]{2}(?=-))',expand=False)
df1['TP_3'] = df1['TP_2'].fillna(df1['TP_1'])



In [45]:
df1 = df1.drop(['TP_1','TP_2'], axis = 1)

In [46]:
df1['TP_3'] = df1['TP_3'].str.upper()


In [47]:
df1

,Jira Ticket,Subject,Product,Location,Priority,Customer Impacted,Date Time,Incident End,First Assignment,Actioned Time,...,Assignee,NAR,RNE,ETS,TSG,RCA,Total Alerts,Incident Start time,Escalated Time,TP_3
0,INC-2640,UDP2 Prod US: Critical - Services Overall - Se...,UDP2,US,P4,No Customer Impact,2022-12-01 08:57:00,2022-12-01 09:02:00,2022-12-01 08:57:00,NaT,...,Ankit Singh,0,NO,0,NO,It was a blip,1,2022-12-01 08:56:00,0,US
1,INC-2649,UDP2 Prod - prod-us-5g-udp2-1: Kafka Consumer ...,UDP2,US,P4,No Customer Impact,2022-12-02 16:15:00,2022-12-02 17:28:00,2022-12-02 16:16:00,2022-12-02 17:23:00,...,Varsha Rana,0,NO,1,YES,SME updated the monitor,2,2022-12-02 16:15:00,0,US
2,INC-2657,Clarity Prod JP: CRITICAL - P3 - 5% Subjects w...,CLARITY,JP,P3,24 users (28.23% impacted),2022-12-05 08:14:00,2022-12-05 12:37:00,2022-12-05 08:14:00,NaT,...,Nitesh Kumar,1,NO,1,YES,It was a blip,1,2022-12-05 08:13:00,2022-12-05 09:24:00,JP
3,INC-2661,Clarity Prod JP: CRITICAL - P4 - 0.5% Subjects...,CLARITY,JP,P4,1 user (0.62% Impacted),2022-12-05 17:37:00,2022-12-05 21:37:00,2022-12-05 17:38:00,NaT,...,Shadab Tamboli,1,NO,0,YES,It was a blip,1,2022-12-05 17:37:00,0,JP
4,INC-2664,UDP2 Prod US: Critical - API Gateway - Unable ...,UDP2,US,P4,No Customer Impact,2022-12-06 06:49:00,2022-12-06 06:59:00,2022-12-06 06:49:00,NaT,...,Pavan K,1,NO,0,YES,It was a blip,1,2022-12-06 06:48:00,0,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,INC-2806,UDP2 Prod EU: Cloudflare 403 Count Above Thres...,UDP2,EU,P4,No Customer Impact,2022-12-21 12:49:00,2022-12-21 13:06:00,2022-12-21 12:50:00,NaT,...,Varsha Rana,1,NO,0,NO,It was a blip,1,NaN,0,EU
98,INC-2808,Clarity Prod EU: CRITICAL - P4 - 0.5% Subjects...,CLARITY,EU,P3,214 users (6.7% Impacted),2022-12-21 13:49:00,2022-12-21 15:26:00,2022-12-21 13:50:00,NaT,...,Varsha Rana,1,NO,1,YES,Pods weren't getting any traffic.,3,NaN,0,EU
99,INC-2811,g5-dexcom-prod-eu-5 - Cassandra: Nodes are O...,UDP,EU,P4,No Customer Impact,2022-12-21 21:32:00,2022-12-21 21:38:00,2022-12-21 21:33:00,NaT,...,Nirmala Patekar,1,NO,0,YES,It was a blip,1,NaN,0,EU
100,INC-2812,g5-dexcom-prod-us-5 - Spark Job nimbus-hourl...,UDP,US,P4,No Customer Impact,2022-12-21 22:10:00,2022-12-21 23:52:00,2022-12-21 22:11:00,2022-12-21 22:16:00,...,Pavan K,0,NO,1,YES,SME has updated the TTL value and restarted th...,1,NaN,0,US


# Location Validation

In [52]:
C = df1.loc[(df1['Location'] != df1['TP_3'])]
print("Check the Subject and Location for the below mentioned tickets to verify :")
C

Check the Subject and Location for the below mentioned tickets to verify :


,Jira Ticket,Subject,Product,Location,Priority,Customer Impacted,Date Time,Incident End,First Assignment,Actioned Time,...,Assignee,NAR,RNE,ETS,TSG,RCA,Total Alerts,Incident Start time,Escalated Time,TP_3
25,INC-2714,Splunk US logs not getting ingested,UDP2,US,P3,No Customer Impact,2022-12-12 10:12:00,2022-12-12 20:29:00,2022-12-12 10:14:00,2022-12-12 11:03:00,...,Nirmala Patekar,0,NO,1,YES,Disabled the HEC Token,10,2022-12-12 10:13:00,2022-12-12 10:19:00,NaN
34,INC-2748,UDP2 Prod MySQL Ent: mysqld process down on o...,UDP2,EU,P4,No Customer Impact,2022-12-15 17:41:00,2022-12-15 17:50:00,2022-12-15 17:42:00,NaT,...,Smruti Sakhare,1,NO,0,YES,It was a blip,8,2022-12-15 17:41:00,0,MYSQL ENT: MYSQLD PROCESS DOWN ON ON HOST
16,INC-2660,UDP2 Prod MySQL Ent: mysqld process down on,UDP2,US,P4,No Customer Impact,2022-12-05 12:26:00,2022-12-05 12:28:00,2022-12-05 12:28:00,NaT,...,Smruti Sakhare,1,NO,0,NO,It was a blip,2,NaN,0,MYSQL ENT
34,INC-2690,udp2 prod us: critical - share realtime patient,UDP2,US,P4,No Customer Impact,2022-12-09 10:06:00,2022-12-09 10:14:00,2022-12-09 10:07:00,NaT,...,Sathya A,1,NO,0,NO,It was a synthetics test,1,NaN,0,NaN
40,INC-2702,UDP2 Prod - prod-us-5g-udp2-1: Kafka Consumer ...,UDP2,-,P4,No Customer Impact,2022-12-10 23:15:00,2022-12-10 23:31:00,2022-12-10 23:16:00,NaT,...,Nitesh Kumar,1,NO,1,YES,It was a blip,2,NaN,0,US
80,INC-2768,UDP2 Prod - : MySQL Ent: Load Exception,UDP2,EU,P4,No Customer Impact,2022-12-18 00:31:00,2022-12-18 00:48:00,2022-12-18 00:36:00,NaT,...,Jainil Lalaji,1,NO,0,NO,It was a blip,2,NaN,0,- : MYSQL ENT
